In [23]:
# --------------------------------------------------
# デフォルトプロンプト　（例）ソフトウェア開発用
# --------------------------------------------------
from openai.types.responses import (
    EasyInputMessageParam,      # 基本の入力メッセージ
    ResponseInputTextParam,     # 入力テキスト
    ResponseInputImageParam,    # 入力画像
    ResponseFormatTextJSONSchemaConfigParam,  # Structured output 用
    ResponseTextConfigParam,    # Structured output 用
    FunctionToolParam,          # 関数呼び出しツール
    FileSearchToolParam,        # ファイル検索ツール
    WebSearchToolParam,         # Web 検索ツール
    ComputerToolParam,          #
    Response
)
def get_default_messages() -> list[EasyInputMessageParam]:
    developer_text = (
        "You are a strong developer and good at teaching software developer professionals "
        "please provide an up-to-date, informed overview of the API by function, then show "
        "cookbook programs for each, and explain the API options."
        "あなたは強力な開発者でありソフトウェア開発者の専門家に教えるのが得意です。"
        "OpenAIのAPIを機能別に最新かつ詳細に説明してください。"
        "それぞれのAPIのサンプルプログラムを示しAPIのオプションについて説明してください。"
    )
    user_text = (
        "Organize and identify the problem and list the issues. "
        "Then, provide a solution procedure for the issues you have organized and identified, "
        "and solve the problems/issues according to the solution procedures."
        "不具合、問題を特定し、整理して箇条書きで列挙・説明してください。"
        "次に、整理・特定した問題点の解決手順を示しなさい。"
        "次に、解決手順に従って問題・課題を解決してください。"
    )
    assistant_text = "OpenAIのAPIを使用するには、公式openaiライブラリが便利です。回答は日本語で"

    return [
    EasyInputMessageParam(role="developer", content=developer_text),
    EasyInputMessageParam(role="user",      content=user_text),
    EasyInputMessageParam(role="assistant", content=assistant_text),
]


In [24]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="OpenAIのresponses.create APIの使い方を教えて。",
)
print(response.output[0].content[0].text)
print('----------------')

fetched_response = client.responses.retrieve(response_id=response.id)

print(fetched_response.output[0].content[0].text)
print('----------------')

response_two = client.responses.create(
    model="gpt-4o-mini",
    input="OpenAIのresponses.parse APIの使い方を教えて。",
    previous_response_id=response.id
)
print(response_two.output[0].content[0].text)
print('----------------')

現在のところ、OpenAIのAPIを使用してテキストを生成する方法について説明します。具体的な `responses.create` メソッドは存在しないため、一般的なAPIの使用方法を説明します。

### OpenAI APIの基本的な使い方

1. **APIキーの取得**:
   OpenAIのウェブサイトでアカウントを作成し、APIキーを取得します。

2. **環境の準備**:
   Pythonを使う場合、必要なライブラリをインストールします。例えば、`openai`ライブラリを使います。

   ```bash
   pip install openai
   ```

3. **APIを呼び出す**:
   以下は、GPT-4を使ってテキストを生成する基本的な例です。

   ```python
   import openai

   # APIキーを設定
   openai.api_key = 'YOUR_API_KEY'

   # テキストを生成するリクエスト
   response = openai.ChatCompletion.create(
       model="gpt-4",
       messages=[
           {"role": "user", "content": "AIについて教えて？"}
       ]
   )

   # 生成された内容を出力
   print(response['choices'][0]['message']['content'])
   ```

### 各パラメータの説明

- `model`: 使用するモデルを指定します（例: `"gpt-4"`）。
- `messages`: 会話の履歴をリスト形式で渡します。`{"role": "user", "content": "..."}`の形式でユーザーのメッセージを追加します。

### 注意点

- **料金**: APIの使用は有料ですので、事前に料金体系を確認しておきましょう。
- **リクエストの制限**: 同時に送信できるリクエスト数やトークン数には制限があります。

詳細なドキュメントや最新情報は[OpenAIの公式ドキュメント](https://platform.openai.com/docs)を参照してくだ

In [30]:
from pydantic import BaseModel

def get_default_messages() -> list[EasyInputMessageParam]:
    developer_text = (
        "You are a strong developer and good at teaching software developer professionals "
        "please provide an up-to-date, informed overview of the API by function, then show "
        "cookbook programs for each, and explain the API options."
        "あなたは強力な開発者でありソフトウェア開発者の専門家に教えるのが得意です。"
        "OpenAIのAPIを機能別に最新かつ詳細に説明してください。"
        "それぞれのAPIのサンプルプログラムを示しAPIのオプションについて説明してください。"
    )
    user_text = (
        "Organize and identify the problem and list the issues. "
        "Then, provide a solution procedure for the issues you have organized and identified, "
        "and solve the problems/issues according to the solution procedures."
        "不具合、問題を特定し、整理して箇条書きで列挙・説明してください。"
        "次に、整理・特定した問題点の解決手順を示しなさい。"
        "次に、解決手順に従って問題・課題を解決してください。"
    )
    assistant_text = "OpenAIのAPIを使用するには、公式openaiライブラリが便利です。回答は日本語で"

    return [
    EasyInputMessageParam(role="developer", content=developer_text),
    EasyInputMessageParam(role="user",      content=user_text),
    EasyInputMessageParam(role="assistant", content=assistant_text),
]

raw_response = client.responses.create(
    model="gpt-4o-mini",  # or another supported model
    input="最新のopenaiのAPIの responses.parse APIの情報は？",
    tools=[{"type": "web_search"}]
)
# ---------- 3) 構造化パース例 ----------
class APINews(BaseModel):
    headline: str
    url: str

messages = get_default_messages()
append_user_text = "上の回答をヘッドラインとURLだけ JSON で返して"
messages.append(
    EasyInputMessageParam(
        role="user",
        content=[
            ResponseInputTextParam(type="input_text", text=raw_response.output_text),
            ResponseInputTextParam(type="input_text", text=append_user_text),
        ]
    )
)

structured = client.responses.parse(
    model="gpt-4o-mini",
    input=messages,
    text_format=APINews
)
import pprint

print(response.output_text)
print('----------------------------------\n')

pprint.pprint(response.model_dump())



response.output_text----------------------------------
現在のところ、OpenAIのAPIを使用してテキストを生成する方法について説明します。具体的な `responses.create` メソッドは存在しないため、一般的なAPIの使用方法を説明します。

### OpenAI APIの基本的な使い方

1. **APIキーの取得**:
   OpenAIのウェブサイトでアカウントを作成し、APIキーを取得します。

2. **環境の準備**:
   Pythonを使う場合、必要なライブラリをインストールします。例えば、`openai`ライブラリを使います。

   ```bash
   pip install openai
   ```

3. **APIを呼び出す**:
   以下は、GPT-4を使ってテキストを生成する基本的な例です。

   ```python
   import openai

   # APIキーを設定
   openai.api_key = 'YOUR_API_KEY'

   # テキストを生成するリクエスト
   response = openai.ChatCompletion.create(
       model="gpt-4",
       messages=[
           {"role": "user", "content": "AIについて教えて？"}
       ]
   )

   # 生成された内容を出力
   print(response['choices'][0]['message']['content'])
   ```

### 各パラメータの説明

- `model`: 使用するモデルを指定します（例: `"gpt-4"`）。
- `messages`: 会話の履歴をリスト形式で渡します。`{"role": "user", "content": "..."}`の形式でユーザーのメッセージを追加します。

### 注意点

- **料金**: APIの使用は有料ですので、事前に料金体系を確認しておきましょう。
- **リクエストの制限**: 同時に送信できるリクエスト数やトークン数には制限があります。

詳細なドキュメントや最新情報は[O